## ※ 데이터나 머신러닝을 관리하기 위한 행동
- Experiment Data Management(실험에 사용되는 데이터 관리)
- Experiment Result management(실험 결과 관리)
  - (어떤 알고리즘, 파라미터를 사용하여 어떤 지표가 나오는지) 
- Best Model Service Deploy(좋은 결과가 나오는 모델 서비스 배포)

#### ■ Data Store : 데이터들을 체계적으로 쌓아놓는 곳



## Experiment Service Deploy, Data store
---
    import numpy as np
    from azureml.core import Dataset

* X_train의 내용을 feature.csv 파일로 저장, 구별자는 ,를 사용
      np.savetxt('feature.csv', X_train, delimiter=',')
* y_train의 내용을 labels.csv 파일로 저장, 구별자는 ,를 사용
      np.savetxt('labels.csv', y_train, delimiter=',')

* 현재 작업공간의 데이터스토어에 대한 정보를 가져오기
      datastore = ws.get_default_datastore()

* 정보를 가져온 데이터 스토어에 파일 업로드
- 업로드할 파일의 경로와 함께 리스트 형식으로 입력, 업로드할 경로 설정, 덮어쓰기 여부 설정
      datastore.upload_files(files=['./feature.csv', './labels.csv'], target_path='diabetes-experiment/', overwrite=True)


* 데이터스토어에 저장했던 데이터 가져오기
       Dataset.Tabular.from_delimited_files(path=[(경로,'경로/파일이름')]) : 구별자가 사용된 파일 가져오기
* 저장되어있는 데이터에 접근할 때 API 안에서 해당되는 파일에 접근 가능, 파일명을 규칙적으로 만들어두면 규칙적 접근 가능
      feature_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/feature.csv')])
      label_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, 'diabetes-experiment/labels.csv')])
---
    import sklearn
    from azureml.core import Mode
    # 모델을 관리하기위한 애져머신러닝 패키지의 모델모듈	
    from azureml.core.resource_configuration import ResourceConfiguration
    # 모델등록을 위한 모델정보 불러오는 모듈
---
## 모델 등록
      model = Model.register(workspace=ws
                model_name='diabetes-experiment-model',
                model_path=f'./{str(best_run.get_file_names()[0])}',		# f는 포맷을 표현한다는 의미 
                model_framework=Model.Framework.SCIKITLEARN,
                model_framework_version=sklearn.__version__,
                sample_input_dataset=feature_dataset,
                sample_output_dataset=label_dataset,
                resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                description='Ridge regression model to predict diabetes progression',
                tags={'area' : 'diabetes', 'type' : 'regression'}
                )
    # 어떠한 과정을 거쳐 나온 모델인지 정보를 전부 적어주기
    # (작업공간, 저장된 모델명, 가장 좋은 실험결과 모델의 경로(String), 모델 학습에 사용한 프레임워크,
    프레임워크 버전, 학습에 사용된 입력데이터(Feature), 출력데이터(Label),
    모델 실행환경 지정(컴퓨터 조립과 유사, 사용할 컴퓨팅자원 지정), 모델에 대한 설명, 모델관리를 위한 태그달기(딕셔너리 타입으로 입력) 

---
    print('Model name : ', model.name)
    print('Model version : ', model.version)
---
### 모델의 배포
    service_name = 'diabetes-service'
    # 서비스 이름 지정

    service = Model.deploy(ws, service_name, [model], overwrite=True)
    # 모델배포, 여러개 지정 가능(리스트타입) 덮어쓰기 여부 지정

    service.wait_for_deployment(show_output=True)
    # 서비스에 배포하는 동안 기다리는데 진행상황을 출력

### Service Deployment Progress
> Creating Container Registry\
-> Registering the environment\
-> Uploading autogenerated assets for no-code-deployment\
-> Building Docker image\
-> Generating deployment configuration\
-> Submitting deploment to compute\
-> Checking the status of deployment service\
-> Checking the status of inference endpoint service

---
    import json

    # json 데이터(훈련용 데이터를 슬라이싱하여 리스트타입으로 넘기고 예측 method 사용)
    input_payload = json.dumps({
        'data': X_train[0:2].values.tolist(),
        'method': 'predict'
    })

    # json 데이터를 실행하여 output에 출력
    output = service.run(input_payload)


    print(output)

---

# Deep Learning
* 신경망
      전기적 신호가 들어가면 어느 정도의 자극을 받으면 다음 세포로 자극을 넘기는 개념.
ex) 수정체에 의해 상이 맺히게 되면 망막세포가 빛을 받아들여 굴절시켜 받아서 대뇌에서 다시 정상형태로 인식.
  - 입력 신호에는 가중치(특정 자극에 대해 중요성을 부여)가 붙어있어 유추해서 움직일 수 있게 한다?
  - 입력 신호값과 가중치값을 전부 더해 어느 값 이상이 되면 활성화 되어 활성 함수가 된다.

## Artificial Neuron and Artificial Neural Network(인공신경과 인공신경망)
- 노드를 볼 때 입력층(Input layer)과 출력층(Output layer) 사이에 실제로 계산하는 층은 Hidden layer.
- 뉴런(=엣지, 세포)의 개수가 많다는 건 데이터(정보량)가 큰 문제를 해결할 수 있다는 의미
- 각각의 층이 많다는 건 깊은 문제를 해결할 수 있다는 의미

### 완전 연결 계층(Fully-connected Layer)
- 가장 기본적인 신경망, Dense Layer라고 불림\
촘촘하게 연결되어있고, 연결 하나하나가 파라미터라 전부 계산해줘야됨.

- 학습 : 신경망에 가중치를 셋팅해놓는 것


#### 신경망의 활용 - 회귀(Regression), 분류(Classfication)

* 텐서(Tensor) : 신경망에 데이터를 흘려보내는 개념.\
가중치 w를 셋팅, 가중치들이 접목이 되면서 텐서에 대한 가장 좋은 가중치가 셋팅됨.\
이 문제를 가장 잘 풀 수 있는 가중치가 셋팅되어있는 신경망 자체가 모델이 됨.
  - 텐서를 표현할수있는 가장 기본적인 방법 = np.ndarray형태
- 스칼라(0차원 텐서) : 방향 X , 차원 X, 값만 있음.
  - 차원이 달라도 연산은 이루어진다.
  - 차원이 부족하면 가상의 차원을 만들어 본차원에 있던 값이 가상의 차원에 있다고 치고 연산한다
        브로드캐스팅 : 다른 형상들끼리 계산이 가능
* 3차원 텐서(이미지 한 장)
      [w, h, [R, G, B]]
  - CNN(Convolutional Neural Network, 합성곱 신경망) : 이미지 처리때 사용
  - 3차원 텐서는 시계열 데이터에도 사용한다.
  - 시계열 데이터를 표현하는 신경망은 따로 있다. (RNN)
* 4차원 텐서(영상)
      [w, h, f, [R, G, B]]
* 5차원 텐서
Color Image Datasets(4차원, 이미지 여러장)
      (samples, height, width, channels) (Keras, Tensorflow)
      (samples, channels, height, width) (Pytorch)
